In [1]:
import numpy as np
import pandas as pd

In [29]:
df=pd.read_excel('nf.xlsx')

In [30]:
df.drop('Unnamed: 0',1,inplace=True)

In [31]:
df1=df.drop([ 'Estimated Annual kWh Savings','First Year Energy Savings $ Estimate'],1)

In [2]:
import math as m
from pso import PSO,Particle
from bga import BGA
from sklearn.ensemble import RandomForestRegressor

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import r2_score
#from bga import BGA
from sklearn.linear_model import Lasso


def Fitness(arr , x_train, y_train , x_validate , y_validate):
    index = [i for i, e in enumerate(arr) if e == 1]
    index = np.array(index)
  
    x_train_new = x_train.iloc[:,index] 
    print(x_train_new.shape)
    x_val_new = x_validate.iloc[:,index]
   
    #regressor=Lasso(alpha=0.005,max_iter=100,normalize=False,tol=0.0005)
    
    #Lasso(alpha=1.0,max_iter=1000,normalize=False,tol=0.0001,fit_intercept=True,precompute=False,copy_X=True,warm_start=False,positive=False,random_state=None,selection='cyclic')
    regressor=RandomForestRegressor(max_depth=80, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=100)
    regressor.fit(x_train_new,y_train)
    #DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
     #                   max_features=2, max_leaf_nodes=None,
      #                  min_impurity_decrease=0.0, min_impurity_split=None,
       #                 min_samples_leaf=1, min_samples_split=2,
        #                min_weight_fraction_leaf=0.0, presort='deprecated',
         #               random_state=None, splitter='best')
    yp=regressor.predict(x_val_new)
    mse = mean_squared_error(y_validate,yp)
    Rmse=m.sqrt(mse)
    AIC=(34754)*m.log(Rmse)+2*len(index)
    print(AIC)
    return  AIC 



train,validate,test=np.split(df1.sample(frac=1),[int(.6*len(df1)),int(.8*len(df1))])
x_train=train.drop('Estimated Annual MMBtu Savings',1)
y_train=train.loc[:,'Estimated Annual MMBtu Savings']
x_test=test.drop('Estimated Annual MMBtu Savings',1)
y_test=test.loc[:,'Estimated Annual MMBtu Savings']
x_validate=validate.drop('Estimated Annual MMBtu Savings',1)
y_validate=validate.loc[:,'Estimated Annual MMBtu Savings']

y_train = y_train.ravel()
y_test = y_test.ravel()

print('Training dataset shape:', x_train.shape, y_train.shape)
print('Testing dataset shape:', x_test.shape, y_test.shape)
print('Validation dataset shape:', x_validate.shape, y_validate.shape)
print(df1.shape)






# call BGA
#test = BGA( x_train, y_train , x_test , y_test, pop_shape=(num_pop, problem_dimentions), method=Fitness, p_c=0.8, p_m=0.2, maxiter = 100, early_stop_rounds=None, verbose = None, maximum=False)

# Run BGA
#best_solution, best_fitness = test.run()

Training dataset shape: (34754, 80) (34754,)
Testing dataset shape: (11585, 80) (11585,)
Validation dataset shape: (11585, 80) (11585,)
(57924, 81)


In [22]:
problem_dimentions = 80
num_pop = 10
maxiter = 5

In [23]:
test = BGA( x_train, y_train , x_test , y_test, pop_shape=(num_pop, problem_dimentions), method=Fitness, p_c=0.8, p_m=0.2, maxiter = 10, early_stop_rounds=None, verbose = None, maximum=False)


In [24]:
best_solution, best_fitness = test.run()

(34754, 29)
113531.87512466336
(34754, 43)
107989.96069625649
(34754, 36)
111519.80393044736
(34754, 39)
108691.49615582512
(34754, 39)
107223.77465863366
(34754, 45)
106756.1679598077
(34754, 43)
108800.44670490625
(34754, 42)
106697.47659713186
(34754, 42)
110390.10077692602
(34754, 39)
108664.35917216617
iteration:  0
(34754, 36)
111527.79830421678
(34754, 36)
111508.42658466384
(34754, 38)
110001.39458044899
(34754, 34)
111875.19297324998
(34754, 39)
108733.74749379375
(34754, 43)
107812.08302913283
(34754, 40)
108456.97879675253
(34754, 42)
106312.55671252662
(34754, 38)
110465.70142948789
(34754, 40)
107413.68380235868
iteration:  1
(34754, 38)
110367.02578215518
(34754, 39)
108641.07184013409
(34754, 39)
111050.29333326737
(34754, 37)
107548.4157452611
(34754, 40)
108550.63853106477
(34754, 43)
107913.07767505363
(34754, 40)
107441.07844790816
(34754, 36)
111505.19211183683
(34754, 41)
106325.71274561457
(34754, 44)
107688.7401916781
iteration:  2
(34754, 42)
108518.11223396969


In [27]:
best_feature_selected = [i for i, e in enumerate(best_solution) if e == 1]
x_train_new = x_train.iloc[:,best_feature_selected] 
print(x_train_new.shape)
# x_val_new = x_validate.iloc[:,index]
  


regressor=RandomForestRegressor(max_depth=80, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=100)
regressor.fit(x_train_new,y_train)


x_test_new = x_test.iloc[:,best_feature_selected]
print(x_test_new.shape)
yp=regressor.predict(x_test_new)
mse = mean_squared_error(y_test,yp)
Rmse=m.sqrt(mse)
AIC=(34754)*m.log(Rmse)+2*38
print(Rmse)
print(AIC)

(34754, 39)
(11585, 39)
20.85441801172746
105643.56253765083


In [13]:
x_train_new.columns

Index([                                                   'Total Project Cost',
                                                            'Total Incentives',
                                             'Amount Financed Through Program',
                                                             'Year Home Built',
                                                                'Size of Home',
                                                              'Volume of Home',
                                                             'Number of Units',
                                                               'Customer Type',
                                                            'Home Performance',
        'Homeowner Received Green Jobs-Green NY Free/Reduced Cost Audit (Y/N)',
                                                         'Project County_Erie',
                                                  'Project County_Westchester',
                                        